# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [21]:
import pandas as pd
import numpy as np
from sklearn import cross_validation, datasets, metrics, tree
import math

In [11]:
boston = sklearn.datasets.load_boston()
print boston.data.shape
print boston.data
print boston.target.shape
print boston.target

(506L, 13L)
[[  6.32000000e-03   1.80000000e+01   2.31000000e+00 ...,   1.53000000e+01
    3.96900000e+02   4.98000000e+00]
 [  2.73100000e-02   0.00000000e+00   7.07000000e+00 ...,   1.78000000e+01
    3.96900000e+02   9.14000000e+00]
 [  2.72900000e-02   0.00000000e+00   7.07000000e+00 ...,   1.78000000e+01
    3.92830000e+02   4.03000000e+00]
 ..., 
 [  6.07600000e-02   0.00000000e+00   1.19300000e+01 ...,   2.10000000e+01
    3.96900000e+02   5.64000000e+00]
 [  1.09590000e-01   0.00000000e+00   1.19300000e+01 ...,   2.10000000e+01
    3.93450000e+02   6.48000000e+00]
 [  4.74100000e-02   0.00000000e+00   1.19300000e+01 ...,   2.10000000e+01
    3.96900000e+02   7.88000000e+00]]
(506L,)
[ 24.   21.6  34.7  33.4  36.2  28.7  22.9  27.1  16.5  18.9  15.   18.9
  21.7  20.4  18.2  19.9  23.1  17.5  20.2  18.2  13.6  19.6  15.2  14.5
  15.6  13.9  16.6  14.8  18.4  21.   12.7  14.5  13.2  13.1  13.5  18.9
  20.   21.   24.7  30.8  34.9  26.6  25.3  24.7  21.2  19.3  20.   16.6
  14.4  

In [15]:
X_train = boston.data[:400]
print X_train.shape
print X_train
X_test = boston.data[400:]
print X_test.shape
print X_test
Y_train = boston.target[:400]
print Y_train.shape
print Y_train
Y_test = boston.target[400:]
print Y_test.shape
print Y_test

(400L, 13L)
[[  6.32000000e-03   1.80000000e+01   2.31000000e+00 ...,   1.53000000e+01
    3.96900000e+02   4.98000000e+00]
 [  2.73100000e-02   0.00000000e+00   7.07000000e+00 ...,   1.78000000e+01
    3.96900000e+02   9.14000000e+00]
 [  2.72900000e-02   0.00000000e+00   7.07000000e+00 ...,   1.78000000e+01
    3.92830000e+02   4.03000000e+00]
 ..., 
 [  7.67202000e+00   0.00000000e+00   1.81000000e+01 ...,   2.02000000e+01
    3.93100000e+02   1.99200000e+01]
 [  3.83518000e+01   0.00000000e+00   1.81000000e+01 ...,   2.02000000e+01
    3.96900000e+02   3.05900000e+01]
 [  9.91655000e+00   0.00000000e+00   1.81000000e+01 ...,   2.02000000e+01
    3.38160000e+02   2.99700000e+01]]
(106L, 13L)
[[  2.50461000e+01   0.00000000e+00   1.81000000e+01 ...,   2.02000000e+01
    3.96900000e+02   2.67700000e+01]
 [  1.42362000e+01   0.00000000e+00   1.81000000e+01 ...,   2.02000000e+01
    3.96900000e+02   2.03200000e+01]
 [  9.59571000e+00   0.00000000e+00   1.81000000e+01 ...,   2.02000000e+

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [16]:
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

In [32]:
base_algorithms_list = []
coefficients_list = []

estimator = tree.DecisionTreeClassifier(random_state = 42, max_depth = 5)
estimator.fit(X_train, [0]*len(Y_train))
base_algorithms_list.append(estimator)
coefficients_list.append(0.9)

for i in range(49):
    estimator = tree.DecisionTreeRegressor(random_state = 42, max_depth = 5)
    #print Y_train
    #print gbm_predict(X_train)
    #print np.asarray(Y_train - gbm_predict(X_train))
    estimator.fit(X_train, Y_train - gbm_predict(X_train))
    base_algorithms_list.append(estimator)
    coefficients_list.append(0.9)

print gbm_predict(X_test)
print Y_test
mse = metrics.mean_squared_error(Y_test, gbm_predict(X_test))
rmse = math.pow(mse, 0.5)
print rmse

with open("ans2.txt", "w") as text_file:
    text_file.write("%s" % rmse)

[12.826253662116782, 11.419687216415017, 12.132572865849022, 10.55276272552009, 9.01674021172939, 8.3019157429331205, 10.668209794934141, 45.004681844645724, 9.912898257420764, 44.1807487590901, 44.867157218757924, 12.764440607521339, 10.799787938036769, 10.004928878088325, 14.067070265273818, 18.029910291438576, 18.361873504088102, 6.905700856296634, 21.090195624318003, 22.722766906432607, 11.851930162806479, 18.591407889653418, 18.483336472642229, 20.82574175311445, 10.677006745164265, 17.975614274101943, 10.852402567764411, 13.282544147424449, 20.808617065387423, 21.790625576066191, 13.45605595502404, 35.443234636830198, 21.937350656423135, 12.821605193491333, 13.479542619682908, 19.643660998589745, 13.261927189954857, 19.453046192950112, 16.048011038574757, 18.469900499123593, 17.717037078042218, 20.820035243192802, 15.72585357599929, 20.191608163644904, 16.694333269200349, 23.520272162814567, 15.501030740456674, 11.908495691347596, 14.730887050652244, 19.540628784104012, 14.908392

In [33]:
len(base_algorithms_list)

50

## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [34]:
base_algorithms_list = []
coefficients_list = []

estimator = tree.DecisionTreeClassifier(random_state = 42, max_depth = 5)
estimator.fit(X_train, [0]*len(Y_train))
base_algorithms_list.append(estimator)
coefficients_list.append(0.9)

for i in range(49):
    estimator = tree.DecisionTreeRegressor(random_state = 42, max_depth = 5)
    estimator.fit(X_train, Y_train - gbm_predict(X_train))
    base_algorithms_list.append(estimator)
    coefficients_list.append(0.9 / (1.0 + i + 1.0))

print gbm_predict(X_test)
print Y_test
mse = metrics.mean_squared_error(Y_test, gbm_predict(X_test))
rmse = math.pow(mse, 0.5)
print rmse

with open("ans3.txt", "w") as text_file:
    text_file.write("%s" % rmse)

[10.828595387632868, 11.7882395460775, 11.855421611298707, 9.2874669510953485, 7.1178994323213702, 7.5953602148300394, 11.737891676833518, 32.890060015476003, 11.031905445063719, 29.784214180550617, 42.456822247852024, 12.370815494400347, 9.5112185333496342, 9.9754603903664307, 8.0406561503812721, 12.08417456005251, 13.882263279774877, 9.2376588760768499, 10.850820472719059, 13.070453884241726, 15.16843135479021, 17.20406020214185, 18.499194026102366, 14.369713881300839, 13.490344688614284, 12.487687961056809, 13.316415495429929, 14.860204359339075, 12.22153450733736, 13.484763287023325, 15.515253227356075, 22.668286686318282, 19.892259665546494, 16.108941972223914, 15.43484119329883, 13.594676794611738, 15.047957408801285, 12.375676627159365, 11.360048964129655, 11.1217831220475, 10.539809186050705, 13.864439322462408, 16.024958692148129, 17.345314619560636, 11.024574668941815, 14.497085098880666, 16.579934841204359, 15.89566974768176, 15.814693255570759, 13.879241831758938, 16.546091

## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.